In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from data_preprocess import DataPreprocessor
from sklearn.preprocessing import StandardScaler

In [3]:
airline_delay_df = pd.read_csv('../Data/flights_sample_3m.csv')
airline_codes_df = pd.read_csv('../Data/airline_codes.csv')
airline_fleet_df = pd.read_csv('../Data/Fleet Data.csv')
airline_prices_df = pd.read_csv('../Data/consumer_airfare.csv')
# busiestairports_df = pd.read_csv('data/busiestAirports.csv', encoding='cp1252')
# passenger_satisfaction_df = pd.read_excel('data/passenger_satisfaction.xlsx')
storm_events_df = pd.read_csv('../Data/StormEvents_details-ftp_v1.0_d2020_c20240620.csv')
international_report_departures_df = pd.read_csv('../Data/International_Report_Departures.csv')
# international_report_passengers_df = pd.read_csv('../Data/international_report_passengers.csv')

In [4]:
state_dict = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "DISTRICT OF COLUMBIA": "DC",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISIANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "PUERTO RICO": "PR",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "VIRGIN ISLANDS": "VI",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY"
}


In [5]:
# storm_events_df['year'] = storm_events_df['BEGIN_YEARMONTH'].astype(str).str[:4].astype(int)
# storm_events_df['month'] = storm_events_df['BEGIN_YEARMONTH'].astype(str).str[4:6].astype(int)
# storm_events_df['day'] = storm_events_df['BEGIN_DAY'].astype(int)
# storm_events_df['date'] = pd.to_datetime(storm_events_df[['year', 'month', 'day']])
# storm_events_df.drop(columns=['year', 'month', 'day'], inplace=True)

In [6]:
# international_report_departures_df['data_dte'] = international_report_departures_df['data_dte'].astype(str)
# international_report_departures_df['date'] = pd.to_datetime(international_report_departures_df['data_dte'])

international_report_departures_df['date'] = pd.to_datetime(
    international_report_departures_df['data_dte'], format="%Y-%m-%d"
)


In [7]:
storm_events_df['date'] = pd.to_datetime(storm_events_df['BEGIN_DATE_TIME'], format="%Y-%m-%d %H:%M:%S")

In [8]:
airline_delay_df['FL_DATE'] = pd.to_datetime(airline_delay_df['FL_DATE'], format='%Y-%m-%d')
airline_delay_df['year'] = airline_delay_df['FL_DATE'].dt.year
international_report_departures_df['year'] = international_report_departures_df['date'].dt.year

In [9]:
airline_delay_df_2020 = airline_delay_df[airline_delay_df['FL_DATE'].dt.year == 2020]
international_report_departures_df_2020 = international_report_departures_df[international_report_departures_df['date'].dt.year == 2020]

In [10]:
airline_delay_df_2020.columns

Index(['Unnamed: 0', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year'],
      dtype='object')

In [11]:
airline_delay_df_2020.fillna(0, inplace=True) 

/var/folders/vd/dmj076xx2t38kz00wdhlwgqh0000gn/T/ipykernel_92677/2125564179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airline_delay_df_2020.fillna(0, inplace=True)


In [12]:
merged_df = airline_delay_df_2020.merge(
    international_report_departures_df_2020,
    left_on=['FL_DATE', 'AIRLINE_CODE'],   # List format for multiple columns
    right_on=['date', 'carrier'],         # List format for multiple columns
    how='left'
)


In [13]:
merged_df['state'] = merged_df['DEST_CITY'].str.split(", ").str[1]
storm_events_df['STATE'] = storm_events_df['STATE'].map(state_dict)

In [14]:
storm_events_df.columns

Index(['Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE', 'EVENT_TYPE',
       'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'date'],
      dtype='object')

In [15]:
merged_df = merged_df.merge(
    storm_events_df,
    left_on=['FL_DATE', 'state'],   # List format for multiple columns
    right_on=['date', 'STATE'],         # List format for multiple columns
    how='inner'
)


In [16]:
# Selecting relevant columns
needed_columns = [
'FL_DATE', 'AIRLINE_CODE', 'state','CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'
]


In [17]:
merged_df['state'].head(1)

0    TX
Name: state, dtype: object

In [18]:
# merged_df = merged_df[needed_columns]

In [19]:
# # Drop rows where essential numerical columns are missing
# airline_fleet_df.dropna(subset=['Current', 'Total', 'Average Age'], inplace=True)

# # Fill missing values in 'Orders' and 'Future' with 0
# airline_fleet_df[['Orders', 'Future']] = airline_fleet_df[['Orders', 'Future']].fillna(0)

# # Fill missing categorical values with "Unknown"
# airline_fleet_df[['Parent Airline', 'Airline', 'Aircraft Type']] = airline_fleet_df[
#     ['Parent Airline', 'Airline', 'Aircraft Type']
# ].fillna('Unknown')

# # Convert cost columns to numeric by removing '$' and ','
# airline_fleet_df['Unit Cost'] = (
#     airline_fleet_df['Unit Cost'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# )
# airline_fleet_df['Total Cost (Current)'] = (
#     airline_fleet_df['Total Cost (Current)'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# )


In [20]:
airline_merged = airline_fleet_df.merge(
    airline_codes_df,
    left_on='Airline',   # List format for multiple columns
    right_on='Airline',         # List format for multiple columns
    how='left'
)


In [21]:
airline_merged = airline_merged.merge(
    airline_prices_df,
    left_on='IATA',   # List format for multiple columns
    right_on='car',         # List format for multiple columns
    how='inner'
)


In [22]:
airline_merged['state'] = airline_merged['city2'].str.split(", ").str[1]

In [23]:
airline_merged.columns

Index(['Unnamed: 0_x', 'Parent Airline', 'Airline', 'Aircraft Type', 'Current',
       'Historic', 'Unit Cost', 'Average Age', 'Unnamed: 0_y', 'IATA', 'ICAO',
       'Unnamed: 0', 'mkt_fare', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'carairlineid', 'car', 'carpax', 'carpaxshare', 'Date',
       'state'],
      dtype='object')

MERGE BELOW AFTER CLEANING

In [24]:
from data_preprocess import DataPreprocessor

In [25]:
# # Selecting relevant columns
# needed_columns = [
# 'IATA', 'Total', 'Orders', 'Unit Cost', 'Total Cost (Current)', 'Average Age','state'
# ]

# airline_merged = airline_merged[needed_columns]


In [26]:
len(merged_df)

3355730

In [27]:
merged_df.columns

Index(['Unnamed: 0_x', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year_x',
       'Unnamed: 0_y', 'data_dte', 'Year', 'Month', 'usg_apt_id', 'usg_apt',
       'usg_wac', 'fg_apt_id', 'fg_apt', 'fg_wac', 'airlineid', 'carrier',
       'carriergroup', 'type', 'Scheduled', 'Charter', 'date_x', 'year_y',
       'state', 'Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE',
       'EVENT_TYPE', 'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'date_y'],
      dtype='object')

In [28]:
# # Selecting relevant columns
# needed_columns = [
# 'FL_DATE', 'AIRLINE_CODE', 'state', 'ARR_DELAY', 'CANCELLED',
#        'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE','INJURIES_DIRECT', 'INJURIES_INDIRECT',
#        'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
#        'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY'
# ]

# merged_df = merged_df[needed_columns]


In [29]:
final_df = DataPreprocessor(merged_df).handle_outliers()

/Users/harshgupta/Desktop/Spring 2025/IDS/Project/cap5771sp25-project/Scripts/data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)
/Users/harshgupta/Desktop/Spring 2025/IDS/Project/cap5771sp25-project/Scripts/data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)
/Users/harshgupta/Desktop/Spring 2025/IDS/Project/cap5771sp25-project/Scripts/data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)


Removing 1701967 rows flagged as outliers across numeric columns.


In [30]:
final_df.columns

Index(['Unnamed: 0_x', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year_x',
       'Unnamed: 0_y', 'data_dte', 'Year', 'Month', 'usg_apt_id', 'usg_apt',
       'usg_wac', 'fg_apt_id', 'fg_apt', 'fg_wac', 'airlineid', 'carrier',
       'carriergroup', 'type', 'Scheduled', 'Charter', 'date_x', 'year_y',
       'state', 'Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE',
       'EVENT_TYPE', 'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'date_y'],
      dtype='object')

In [31]:
# merged_df.to_csv('data/merged_df.csv', index=False)
# airline_merged.to_csv('data/airline_merged.csv', index=False)

In [32]:
null_counts = merged_df.isnull().sum()
print(null_counts)


Unnamed: 0_x                    0
FL_DATE                         0
AIRLINE                         0
AIRLINE_CODE                    0
DOT_CODE                        0
FL_NUMBER                       0
ORIGIN                          0
ORIGIN_CITY                     0
DEST                            0
DEST_CITY                       0
CRS_DEP_TIME                    0
DEP_DELAY                       0
CRS_ARR_TIME                    0
ARR_DELAY                       0
CANCELLED                       0
DIVERTED                        0
CRS_ELAPSED_TIME                0
ELAPSED_TIME                    0
AIR_TIME                        0
DISTANCE                        0
DELAY_DUE_CARRIER               0
DELAY_DUE_WEATHER               0
DELAY_DUE_NAS                   0
DELAY_DUE_SECURITY              0
DELAY_DUE_LATE_AIRCRAFT         0
Month_Year                      0
year_x                          0
Unnamed: 0_y               149531
data_dte                   149531
Year          

In [33]:
final_merged_df = merged_df.dropna(subset=['MAGNITUDE'])

In [34]:
len(final_merged_df)

3355730

In [35]:
# final_merged_df.to_csv('data/final_merged_df.csv', index=False)

In [36]:
len(airline_merged)

60256

In [37]:
airline_merged = airline_merged.drop_duplicates(subset=['Unit Cost', 'IATA'])

In [ ]:
final_merged_df.to_csv("../Data/pre_final_merged.csv")

NameError: name 'final_merged' is not defined

In [40]:
final_merged = final_merged_df.merge(
    airline_merged,
    left_on=['state','AIRLINE_CODE'],   # List format for multiple columns
    right_on=['state','IATA'],         # List format for multiple columns
    how='left'
)

: 

In [40]:
final_merged.columns

Index(['FL_DATE', 'AIRLINE_CODE', 'state', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT',
       'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'MAGNITUDE', 'MAGNITUDE_TYPE',
       'FLOOD_CAUSE', 'CATEGORY', 'IATA', 'Total', 'Orders', 'Unit Cost',
       'Total Cost (Current)', 'Average Age'],
      dtype='object')

In [ ]:
# final_merged.dropna(subset=['FL_DATE','AIRLINE_CODE','state','MAGNITUDE','Average Age'], inplace=True)

In [43]:
len(final_merged)

548849

In [45]:
final_merged['FL_DATE'].unique()

<DatetimeArray>
['2020-07-31 00:00:00', '2020-03-28 00:00:00', '2020-04-29 00:00:00',
 '2020-10-26 00:00:00', '2020-08-19 00:00:00', '2020-11-09 00:00:00',
 '2020-09-28 00:00:00', '2020-07-29 00:00:00', '2020-02-09 00:00:00',
 '2020-06-24 00:00:00',
 ...
 '2020-11-12 00:00:00', '2020-09-19 00:00:00', '2020-02-29 00:00:00',
 '2020-12-28 00:00:00', '2020-06-07 00:00:00', '2020-01-08 00:00:00',
 '2020-04-21 00:00:00', '2020-04-20 00:00:00', '2020-05-14 00:00:00',
 '2020-12-03 00:00:00']
Length: 248, dtype: datetime64[ns]

In [47]:
prod_final = DataPreprocessor(final_merged).handle_outliers()

d:\florida_coursework\second_sem\intro_to_data_science\ids_env\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\florida_coursework\second_sem\intro_to_data_science\ids_env\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Removing 247831 rows flagged as outliers across numeric columns.


In [48]:
len(prod_final)

301018